In [1]:
import pandas as pd
from weblogo import *

### Configuration

In [2]:
sea = "sea"  # change to specific directory of MEME/bin/sea
df_name = "variant_A.csv"  # input, be careful for multiindex
motif_col = "motif_F10"
cluster_col = "Cluster"
meme_fn = "variant.meme"
fasta_fn = "variant.fa"
sea_out = "sea_out"

### Run

In [3]:
df = pd.read_csv(df_name, index_col=None, header=[0])
# df = pd.read_csv(df_name, index_col=[0,1,2], header=[0, 1]) # for dual header

In [4]:
df

,Unnamed: 0,Tan et al,motif_F10,base,num_of_sample,known_mod,X,Y,Cluster
0,chr11@65273630@+,True,ACGGGGTTCAAATCCCTGCGG,A,761,m1A,6.023635,5.492287,1
1,chr1@19654425@+,True,CCTGGGTTCAAATCCTGGCTC,A,328,m1A,6.018350,5.493500,1
2,chr9@100774726@+,True,GACTTGATGAAGAAGATGAAG,A,753,NaN,4.549586,0.021215,2
3,chr9@100774731@+,True,GATGAAGAAGATGAAGATGAG,A,715,NaN,3.728684,-1.285071,3
4,chr17@8129568@-,True,CCTGGGTTCGAATCCCAGCGG,A,238,m1A,6.024044,5.463328,1
...,...,...,...,...,...,...,...,...,...
2517,chr22@38880996@-,True,TATATTCAAAATGGGCAAATT,A,6,NaN,3.428226,2.654001,12
2518,chr2@61721173@-,True,AAATCTTTAAAATTTGTCTTG,A,6,NaN,4.709414,3.585597,14
2519,chr15@77241631@+,True,CTTTTGTGATAAAATATTGAT,A,6,NaN,4.777838,2.031891,11
2520,chr2@64112921@+,True,TGGGTGCCACAGTGGATCTGT,A,6,NaN,2.426064,0.186146,16


In [5]:
def extact_all_fasta(df_in, column, fn_out, rna=True):
    N = 0
    with open(fn_out, "w") as output:
        for idx, row in df_in.iterrows():
            if rna == True:
                output.write(">{}\n{}\n".format(N, row[column].replace("T", "U")))
            else:
                output.write(">{}\n{}\n".format(N, row[column]))
            N += 1        

In [6]:
extact_all_fasta(df, motif_col, fasta_fn)

In [7]:
def generate_meme_file(df_in, id_column, motif_column, fn_out, rna=True):
    all_ids = set(df_in[id_column].tolist())
    temp_mat_name = fn_out+".temp.mat"
    print(all_ids)
    with open(temp_mat_name, "w") as output:
        for ID in all_ids:
            print(ID)
            subdf = df_in[df_in[id_column]==ID]
            count_data = {}
            for _, row in subdf.iterrows():
                if "N" in row[motif_column]:
                    continue
                if rna == True:
                    iterseq = row[motif_column].replace("T", "U")
                else:
                    iterseq = row[motif_column]
                for idx, base in enumerate(list(iterseq)):
                    if idx not in count_data:
                        count_data[idx] = {"A":0, "C": 0, "G": 0, "U":0}
                    count_data[idx][base] += 1
            count_df = pd.DataFrame.from_dict(count_data).T
            seqs = count_df.values
            seqs = np.array(seqs)
            logodata = LogoData.from_counts(counts=seqs, alphabet='ACGU')

            temp = []
            for i in range(logodata.counts.shape[0]):
                # temp.extend(list(logodata.entropy[i]*logodata.counts[i]/logodata.counts[i].sum()))
                output.write("{}\t{}\t{}\t{}\n".format(logodata.counts[i][0], logodata.counts[i][1], logodata.counts[i][2], logodata.counts[i][3]))
            output.write("\n")
        
    !matrix2meme -rna < $temp_mat_name > $fn_out

In [8]:
generate_meme_file(df, cluster_col, motif_col, meme_fn)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [9]:
!$sea -oc $sea_out -p $fasta_fn -m $meme_fn

# Checking alphabets in 1 motif files.
# Loading motifs from file 'variant.meme'
# Alphabet: RNA
# NOTE: Will convert any DNA sequences to RNA.
# Positive sequences "variant.fa" - training: 2270 hold-out: 252
# Negative sequences are shuffled primary sequences (2-order) - training: 2270 hold-out: 252
# Estimating background model from control sequences.
# Background: A 0.292 C 0.252 G 0.233 U 0.224
# Background order: 2 Background size: 84
# Using Fisher Exact test for p-values.
# Computing q-values.
#   Cannot estimate pi_0 accurately from fewer than 100 p-values.
#   Total p-values = 19. Using pi_zero = 1.0.
# Freeing storage...


### It is better to read the html.

In [10]:
df_sea = pd.read_csv("./{}/sea.tsv".format(sea_out), header=0, sep="\t")

In [11]:
df_sea

,RANK,DB,ID,ALT_ID,CONSENSUS,TP,TP%,FP,FP%,ENR_RATIO,SCORE_THR,PVALUE,LOG_PVALUE,EVALUE,LOG_EVALUE,QVALUE,LOG_QVALUE
0,1,variant.meme,15.0,NNNNNNDNHHACHNNNNNNNN,NNNNNNDNHCACHBBNNNNNN,853.0,37.58,249.0,10.97,3.42,0.4500,2.870000e-101,-231.51,5.460000e-100,-228.56,5.460000e-100,-228.56
1,2,variant.meme,19.0,NNNNSNNNNSACCYNNVNBHB,SBSNSBNSBCACCCCVSVSCC,653.0,28.77,163.0,7.18,3.99,0.0560,1.640000e-84,-192.92,3.110000e-83,-189.98,1.560000e-83,-190.67
2,3,variant.meme,16.0,NNBNNNBNNSASNVNVNVNNN,BSBSVBBBSSASVGVSCSBVB,752.0,33.13,242.0,10.66,3.10,0.0280,1.100000e-77,-177.20,2.100000e-76,-174.26,6.990000e-77,-175.35
3,4,variant.meme,9.0,NNNNNHDBNNAVRNDNNNNDN,NNNBMHWYNMARADDBNDRRD,727.0,32.03,307.0,13.52,2.36,0.0086,3.780000e-51,-116.10,7.180000e-50,-113.16,1.800000e-50,-114.54
4,5,variant.meme,17.0,SNNYNBCHSCACYHCYNSBNY,SMCCNCCWSCACCCCCNSCNC,405.0,17.84,98.0,4.32,4.10,0.0850,1.040000e-50,-115.09,1.970000e-49,-112.15,3.940000e-50,-113.76
5,6,variant.meme,18.0,NBHNNYMCYCACCCYYHHCHN,HSYHBCCCYCACCCCCCCCHB,372.0,16.39,88.0,3.88,4.19,0.0200,3.470000e-47,-106.98,6.590000e-46,-104.03,1.100000e-46,-105.83
6,7,variant.meme,12.0,NNNNHNNVAWABNNNNNNNNN,HNNNWNNMAAACNNNNNNDND,467.0,20.57,160.0,7.05,2.91,1.6000,2.630000e-41,-93.44,5.000000e-40,-90.49,7.150000e-41,-92.44
7,8,variant.meme,11.0,DNNNNNNNNNAAADNNDNDDN,DNDNDDWNUDAAADNDWHWDW,628.0,27.67,273.0,12.03,2.30,0.0060,9.070000e-41,-92.20,1.720000e-39,-89.26,2.150000e-40,-91.34
8,9,variant.meme,13.0,NNNNNDNDNHAHNKKUBUNHW,DBWNNWNDWUAYUUUUUUUUU,440.0,19.38,156.0,6.87,2.81,0.0670,5.030000e-37,-83.58,9.550000e-36,-80.64,1.060000e-36,-82.83
9,10,variant.meme,5.0,DNDRNNNRNRARRRNNDBNRD,DRDGRGWGRGAGRGDGRGVGR,390.0,17.18,163.0,7.18,2.38,0.3600,1.400000e-25,-57.23,2.670000e-24,-54.28,2.670000e-25,-56.58
